# Kepler Framework, Emulate v8

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v10.h5'
tpath_data18 = 'output/emulation/data18_table_v10.h5'
tpath_mc16   = 'output/emulation/mc16_table_v10.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v10_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v10_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v10_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v10_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v10" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v10_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v10_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v10_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v10_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v10_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:38<00:00, 24.60s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v10.root')

2022-03-09 23:57:47,456 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v10.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:53<00:00, 22.65s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5184  | 29970371/32047563 |
|   L2   | 93.2773  | 29893095/32047563 |
| EFCalo | 91.9663  | 29472954/32047563 |
|  HLT   | 90.0268  | 28851397/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.49s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2435  | 21455098/22765602 |
|   L2   | 94.1279  | 21428793/22765602 |
| EFCalo | 93.2029  | 21218198/22765602 |
|  HLT   | 87.0220  | 19811086/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.2124  | 1329657/1367786 |
|   L2   | 97.1233  | 1328439/1367786 |
| EFCalo | 95.0318  | 1299831/1367786 |
|  HLT   | 91.7663  | 1255167/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 19.87it/s]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.9947  | 59327/60541  |
|   L2   | 97.9270  | 59286/60541  |
| EFCalo | 96.6056  | 58486/60541  |
|  HLT   | 94.6317  | 57291/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v10.root')

2022-03-10 00:02:09,871 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v10.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname = '!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname = '!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname = '!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname = '!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.48s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  5.2528  | 543664/10349992  |
|   L2   |  4.6522  | 481505/10349992  |
| EFCalo |  4.4736  | 463021/10349992  |
|  HLT   |  0.1506  |  15592/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.96s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.6020  |  55710/9254300  |
|   L2   |  0.5557  |  51422/9254300  |
| EFCalo |  0.4333  |  40099/9254300  |
|  HLT   |  0.0013  |   124/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.35s/it]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.6695  |  52571/7852181  |
|   L2   |  0.6136  |  48178/7852181  |
| EFCalo |  0.4534  |  35598/7852181  |
|  HLT   |  0.0015  |   119/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.52s/it]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  4.2837  |  241345/5634041 |
|   L2   |  3.8283  |  215690/5634041 |
| EFCalo |  3.5601  |  200579/5634041 |
|  HLT   |  0.0463  |   2607/5634041  |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:36<00:00, 24.12s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v10.root')

2022-03-10 00:07:47,610 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v10.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.54s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5627  | 30520601/32620495 |
|   L2   | 93.3248  | 30443013/32620495 |
| EFCalo | 91.9912  | 30008001/32620495 |
|  HLT   | 90.5487  | 29537431/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.47s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2110  | 21964188/23313826 |
|   L2   | 94.1152  | 21941849/23313826 |
| EFCalo | 93.3154  | 21755390/23313826 |
|  HLT   | 87.5955  | 20421857/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.21s/it]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.2300  | 1343403/1381675 |
|   L2   | 97.1418  | 1342184/1381675 |
| EFCalo | 95.0528  | 1313321/1381675 |
|  HLT   | 92.0838  | 1272299/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.37it/s]


HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.0079  | 59186/60389  |
|   L2   | 97.9102  | 59127/60389  |
| EFCalo | 96.7279  | 58413/60389  |
|  HLT   | 94.8898  | 57303/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v10.root')

2022-03-10 00:12:09,823 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v10.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname = '!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname = '!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname = '!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname = '!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:18<00:00,  3.62s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  5.7150  |  483896/8467098 |
|   L2   |  5.0156  |  424672/8467098 |
| EFCalo |  4.8398  |  409790/8467098 |
|  HLT   |  0.6338  |  53661/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.20s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.9145  |  69765/7628804  |
|   L2   |  0.8703  |  66392/7628804  |
| EFCalo |  0.7517  |  57349/7628804  |
|  HLT   |  0.0399  |   3044/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.74s/it]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.9646  |  63300/6561985  |
|   L2   |  0.9108  |  59767/6561985  |
| EFCalo |  0.7579  |  49733/6561985  |
|  HLT   |  0.0480  |   3152/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.10s/it]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  4.7524  |  226997/4776518 |
|   L2   |  4.2378  |  202419/4776518 |
| EFCalo |  3.9772  |  189970/4776518 |
|  HLT   |  0.3232  |  15438/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [25]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [26]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 30.82it/s]


In [27]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v10.root')

2022-03-10 00:13:27,065 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v10.root


In [28]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.38it/s]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 92.8834  | 73024/78619  |
|   L2   | 92.8745  | 73017/78619  |
| EFCalo | 73.1986  | 57548/78619  |
|  HLT   | 72.1416  | 56717/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.90it/s]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 85.7424  | 62682/73105  |
|   L2   | 85.7370  | 62678/73105  |
| EFCalo | 68.7080  | 50229/73105  |
|  HLT   | 66.1049  | 48326/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.35it/s]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 89.3181  | 66040/73938  |
|   L2   | 89.3127  | 66036/73938  |
| EFCalo | 70.3116  | 51987/73938  |
|  HLT   | 68.6007  | 50722/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.44it/s]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 92.1992  | 62654/67955  |
|   L2   | 92.1948  | 62651/67955  |
| EFCalo | 72.0300  | 48948/67955  |
|  HLT   | 70.6203  | 47990/67955  |
+--------+----------+--------------+


In [29]:
eff_mc16.save()